In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data=pd.read_csv('../datasets/testdata.csv',sep=',')
names=data.columns
print(names)

print(len(names))
cell_features=data.iloc[:,1:116].values
y=data.iloc[:,0].values

le=LabelEncoder()
y_true=le.fit_transform(y)

Index(['labelsC2', 'g.s', 'g.p', 'g.pdm', 'g.pdsd', 'g.effr', 'g.acirc',
       'g.sf', 'g.theta', 'g.l1',
       ...
       'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24'],
      dtype='object', length=116)
116


In [2]:
# class distribution
print(data.groupby('labelsC2').size())

labelsC2
a    1075
b     938
c    1508
l     875
o    1069
dtype: int64


In [3]:
#Normalize
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation
test_size=0.3
seed=0

X_train,X_test,Y_train,Y_test = cross_validation.train_test_split(cell_features,y_true,test_size=test_size,random_state=seed)


#Normalize training to improve performance
#we can use minmax scaling or standard scaler based scaling
stdSlr = StandardScaler().fit(X_train)
X_train = stdSlr.transform(X_train)

#Normalize test dataset same as training dataset
stdtestslr=StandardScaler().fit(X_test)
X_test=stdtestslr.transform(X_test)

print(X_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(Y_test)

(3825, 115)
(1640, 115)
(1640,)
[2 2 1 ..., 4 0 1]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
#Load Libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

num_folds = 10
num_instances=len(X_train)
seed = 7

scoring='accuracy'
# ensembles
ensembles = []
ensembles.append(('AB', AdaBoostClassifier()))
ensembles.append(('GBM', GradientBoostingClassifier()))
results = []
names = []
for name, model in ensembles:
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv=kfold,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


AB: 0.751370 (0.029484)
GBM: 0.884437 (0.018675)


In [6]:
print(ensembles)

[('AB', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)), ('GBM', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False))]


In [7]:
kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
cv_result=cross_validation.cross_val_score(GradientBoostingClassifier(), X_train, Y_train, cv=kfold,scoring=scoring)

In [8]:
print(cv_result)

[ 0.86684073  0.89817232  0.91383812  0.87206266  0.89556136  0.85863874
  0.86649215  0.90052356  0.86910995  0.90052356]


In [10]:
GBM_Clf=GradientBoostingClassifier().fit(X_train, Y_train)

In [11]:
test_accuracy_GBM=GBM_Clf.score(X_test,Y_test)
print("Ensemble classifier score on testing dataset is:",test_accuracy_GBM)

Ensemble classifier score on testing dataset is: 0.881707317073


To Do for participants:
Compare the performance of the two boosting classifiers and report the evaluation results in terms of confusion matrix and ROC. Upload your solutions or email us your jupyter Notebook files.